In [7]:

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IMPORT LIBRARIES

In [8]:
import os
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, HistGradientBoostingClassifier
from sklearn.metrics import (roc_auc_score, average_precision_score, accuracy_score,
                             precision_score, recall_score, f1_score, confusion_matrix,
                             classification_report, roc_curve, precision_recall_curve)
import joblib
import math
import time
import json
from tqdm import tqdm

In [9]:
try:
    import xgboost as xgb
    XGBOOST_AVAILABLE = True
except Exception:
    XGBOOST_AVAILABLE = False

try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False


LOAD DATA

In [10]:
DATA_PATH = "/content/drive/MyDrive/final_dataset_realistic 1.csv"
OUTPUT_DIR = "/content/drive/MyDrive/readmission_output.csv"
os.makedirs(OUTPUT_DIR, exist_ok=True)

RANDOM_STATE = 42
TEST_SIZE = 0.2
CV_FOLDS = 3
N_JOBS = -1
SCORING_METRIC = "roc_auc"
RISK_THRESHOLD = 0.5

In [11]:
def find_target_col(df):
    """Find a readmission-like target column name in dataframe."""
    candidates = [c for c in df.columns if any(k in c.lower() for k in ['readmit','readmission','readmitted','readmit_30','readmission_30'])]
    return candidates[0] if candidates else None

def remove_existing_risk_cols(df):
    risk_cols = [c for c in df.columns if 'risk' in c.lower()]
    if risk_cols:
        print("Removing existing risk columns:", risk_cols)
        df = df.drop(columns=risk_cols, errors='ignore')
    return df

def safe_to_numeric(s):
    try:
        return pd.to_numeric(s, errors='coerce')
    except Exception:
        return s

LOAD & CLEAN

In [12]:
print("Loading data from:", DATA_PATH)
df_raw = pd.read_csv(DATA_PATH, dtype=object)
print("Initial shape:", df_raw.shape)
df_raw = remove_existing_risk_cols(df_raw)

# Detect target
target_col = find_target_col(df_raw)
if target_col is None:
    raise RuntimeError("No readmission target column found. Please include a column name containing 'readmit' or 'readmission'.")

print("Detected target column:", target_col)

Loading data from: /content/drive/MyDrive/final_dataset_realistic 1.csv
Initial shape: (5000, 37)
Detected target column: Readmission


In [13]:
df_raw.columns

Index(['Patient Name', 'Admission ID', 'Age', 'Sex', 'Weight',
       'Admission Date', 'Admission Time', 'Consultant Doctor Name',
       'Doctor Name', 'Doctor ID', 'Problem Type', 'Discharge Date',
       'Discharge Time', 'Readmission', 'Blood Pressure', 'Insulin',
       'Blood Group', 'Cholesterol', 'Platelets', 'Diabetics',
       'Problem Description', 'Nurse Name', 'Patient Phone Number',
       'Patient Mail ID', 'weather', 'air_quality_index', 'social_event_count',
       'Hemoglobin (g/dL)', 'WBC Count (10^9/L)', 'Platelet Count (10^9/L)',
       'Urine Protein (mg/dL)', 'Urine Glucose (mg/dL)', 'ECG Result',
       'Pulse Rate (bpm)', 'State', 'City', 'Location'],
      dtype='object')

NORMALIZE THE VALUES

In [14]:
df = df_raw.copy()
df[target_col] = df[target_col].astype(str).str.strip().str.lower().map({
    'yes':'1','y':'1','true':'1','1':'1','no':'0','n':'0','false':'0','0':'0'
})
df[target_col] = pd.to_numeric(df[target_col], errors='coerce')
print("Target value counts (including NaN):")
print(df[target_col].value_counts(dropna=False).to_string())

Target value counts (including NaN):
Readmission
0    3212
1    1788


DROP ROWS WITH MISSING VALUES

In [15]:
before = len(df)
df = df[df[target_col].notna()].copy()
print(f"Dropped {before - len(df)} rows with missing target. Remaining: {len(df)}")

Dropped 0 rows with missing target. Remaining: 5000


EXPLORATORY DATA ANALYTICS

In [16]:
print("\n--- Basic EDA ---")
print("Columns:", list(df.columns))
print("\nNumeric sample summary (attempt coercion):")
# attempt to coerce commonly numeric columns for summary
sample_numeric_cols = []
for c in df.columns:
    coerced = pd.to_numeric(df[c], errors='coerce')
    if coerced.notna().sum() > max(10, 0.01 * len(df)):  # if column has many numeric-like values
        sample_numeric_cols.append(c)
print("Numeric-candidate columns:", sample_numeric_cols[:20])
if 'age' in [c.lower() for c in df.columns]:
    ac = [c for c in df.columns if c.lower()=='age'][0]
    print(df[ac].astype(str).describe())


--- Basic EDA ---
Columns: ['Patient Name', 'Admission ID', 'Age', 'Sex', 'Weight', 'Admission Date', 'Admission Time', 'Consultant Doctor Name', 'Doctor Name', 'Doctor ID', 'Problem Type', 'Discharge Date', 'Discharge Time', 'Readmission', 'Blood Pressure', 'Insulin', 'Blood Group', 'Cholesterol', 'Platelets', 'Diabetics', 'Problem Description', 'Nurse Name', 'Patient Phone Number', 'Patient Mail ID', 'weather', 'air_quality_index', 'social_event_count', 'Hemoglobin (g/dL)', 'WBC Count (10^9/L)', 'Platelet Count (10^9/L)', 'Urine Protein (mg/dL)', 'Urine Glucose (mg/dL)', 'ECG Result', 'Pulse Rate (bpm)', 'State', 'City', 'Location']

Numeric sample summary (attempt coercion):
Numeric-candidate columns: ['Age', 'Weight', 'Readmission', 'Cholesterol', 'Platelets', 'air_quality_index', 'social_event_count', 'Hemoglobin (g/dL)', 'WBC Count (10^9/L)', 'Platelet Count (10^9/L)', 'Urine Protein (mg/dL)', 'Urine Glucose (mg/dL)', 'Pulse Rate (bpm)']
count     5000
unique      72
top        

In [17]:
def quick_plots(df_local, target):
    try:
        sns.set()
        plt.figure(figsize=(5,3))
        sns.countplot(x=target, data=df_local)
        plt.title("Target distribution")
        plt.show()
    except Exception as e:
        print("Skipping quick_plots due to:", e)

In [18]:
print("\n--- Feature engineering ---")
df_fe = df.copy()


--- Feature engineering ---


In [19]:
cols_lower = {c.lower(): c for c in df_fe.columns}

In [20]:
age_col = None
for candidate in ['age','patient_age','age_years']:
    if candidate in cols_lower:
        age_col = cols_lower[candidate]
        break

if age_col:
    df_fe[age_col] = pd.to_numeric(df_fe[age_col], errors='coerce')
    df_fe['age_bucket'] = pd.cut(df_fe[age_col], bins=[0,30,50,65,80,200], labels=['<=30','31-50','51-65','66-80','80+'])
    print("Created age_bucket from", age_col)
else:
    print("No age column found.")

Created age_bucket from Age


In [21]:
admit_col = None
discharge_col = None
for c in df_fe.columns:
    low = c.lower()
    if 'admit' in low and admit_col is None:
        admit_col = c
    if 'discharg' in low and discharge_col is None:
        discharge_col = c

if admit_col:
    df_fe[admit_col] = pd.to_datetime(df_fe[admit_col], errors='coerce')
if discharge_col:
    df_fe[discharge_col] = pd.to_datetime(df_fe[discharge_col], errors='coerce')

if admit_col and discharge_col:
    df_fe['los_days'] = (df_fe[discharge_col] - df_fe[admit_col]).dt.days
    df_fe.loc[(df_fe['los_days'] < 0) | (df_fe['los_days'] > 3650), 'los_days'] = np.nan
    print("Computed los_days from", admit_col, "and", discharge_col)
else:
    print("Admission/discharge dates not both found -> skipping LOS feature.")

# Comorbidity columns detection and count
comorbidity_keywords = ['diabetes','hypertension','hyperten','cancer','copd','asthma','heart','renal','kidney','stroke']
comorb_cols = [c for c in df_fe.columns if any(k in c.lower() for k in comorbidity_keywords)]
print("Detected comorbidity-like columns:", comorb_cols[:20])
for c in comorb_cols:
    df_fe[c] = pd.to_numeric(df_fe[c], errors='coerce').fillna(df_fe[c].astype(str).str.lower().map({'yes':1,'y':1,'true':1,'1':1,'no':0,'n':0,'false':0}))
    df_fe[c] = pd.to_numeric(df_fe[c], errors='coerce').fillna(0).astype(int)
if comorb_cols:
    df_fe['comorbidity_count'] = df_fe[comorb_cols].sum(axis=1)
else:
    df_fe['comorbidity_count'] = 0

# Date-based features (admit weekday/month)
if admit_col:
    df_fe['admit_weekday'] = df_fe[admit_col].dt.weekday
    df_fe['admit_month'] = df_fe[admit_col].dt.month

# Coerce some lab-like columns to numeric if present
lab_keywords = ['hemoglobin','hb','wbc','platelet','creatinine','cholesterol','glucose','pulse']
for c in df_fe.columns:
    if any(k in c.lower() for k in lab_keywords):
        df_fe[c] = pd.to_numeric(df_fe[c], errors='coerce')

# Drop personal-identifying or long-text columns
drop_candidates = [c for c in df_fe.columns if any(k in c.lower() for k in ['name','address','phone','mail','email','notes','description','image','photo','url'])]
if drop_candidates:
    print("Dropping candidate PII / long-text columns (count={}): {}".format(len(drop_candidates), drop_candidates[:10]))
    df_fe = df_fe.drop(columns=drop_candidates, errors='ignore')

print("Feature engineering completed. Shape:", df_fe.shape)

Admission/discharge dates not both found -> skipping LOS feature.
Detected comorbidity-like columns: []
Dropping candidate PII / long-text columns (count=7): ['Patient Name', 'Consultant Doctor Name', 'Doctor Name', 'Problem Description', 'Nurse Name', 'Patient Phone Number', 'Patient Mail ID']
Feature engineering completed. Shape: (5000, 32)


PREPARE FEATURE MATRIX

In [22]:
print("\nPreparing feature matrix X and target y...")
# Ensure target is int
df_fe[target_col] = df_fe[target_col].astype(int)
X = df_fe.drop(columns=[target_col], errors='ignore')
y = df_fe[target_col]

# Identify numeric and categorical features
numeric_cols = X.select_dtypes(include=['int64','float64']).columns.tolist()
# Treat small-cardinality object columns as categorical
cat_cols = [c for c in X.columns if (X[c].dtype == object or X[c].nunique() < 50) and c not in numeric_cols]

# Remove columns that are obviously datetime objects from feature lists
numeric_cols = [c for c in numeric_cols if not np.issubdtype(type(X[c].dtype), np.datetime64)]
# Make sure we don't include admit/discharge date columns directly
for dtcol in [admit_col, discharge_col]:
    if dtcol in cat_cols: cat_cols.remove(dtcol)
    if dtcol in numeric_cols: numeric_cols.remove(dtcol)

print("Numeric cols count:", len(numeric_cols))
print("Categorical cols count:", len(cat_cols))

# Preprocessor
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, cat_cols)
], remainder='drop', sparse_threshold=0)


Preparing feature matrix X and target y...
Numeric cols count: 9
Categorical cols count: 21


SPLIT

In [23]:
print("\nTrain/test split (test_size={})".format(TEST_SIZE))
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SIZE,
                                                    random_state=RANDOM_STATE, stratify=y)
print("Train size:", X_train.shape, "Test size:", X_test.shape)


Train/test split (test_size=0.2)
Train size: (4000, 31) Test size: (1000, 31)


MODEL TRAINING & HYPER TUNING

In [24]:
results = {}

def evaluate_pipeline(name, pipeline, X_test, y_test):
    y_prob = pipeline.predict_proba(X_test)[:,1]
    y_pred = (y_prob >= RISK_THRESHOLD).astype(int)
    metrics = {}
    metrics['roc_auc'] = roc_auc_score(y_test, y_prob)
    metrics['pr_auc']  = average_precision_score(y_test, y_prob)
    metrics['accuracy'] = accuracy_score(y_test, y_pred)
    metrics['precision'] = precision_score(y_test, y_pred, zero_division=0)
    metrics['recall'] = recall_score(y_test, y_pred, zero_division=0)
    metrics['f1'] = f1_score(y_test, y_pred, zero_division=0)
    metrics['confusion_matrix'] = confusion_matrix(y_test, y_pred)
    print(f"\n{name} Evaluation:")
    print(f"ROC-AUC: {metrics['roc_auc']:.4f} | PR-AUC: {metrics['pr_auc']:.4f} | Acc: {metrics['accuracy']:.4f} | F1: {metrics['f1']:.4f}")
    print("Confusion matrix:\n", metrics['confusion_matrix'])
    print("Classification report:\n", classification_report(y_test, y_pred, digits=4))
    return metrics

TRAINING OF DIABETICS AND THE HEART FAILURE

In [28]:
# ============================================================
# Train 3 Models for Patient Readmission:
#   - Logistic Regression
#   - Random Forest
#   - XGBoost
# Separately for:
#   - Diabetes
#   - Heart Failure
# Saves:
#   - readmission_diabetes_RandomForest.pkl
#   - readmission_heart_failure_RandomForest.pkl
# ============================================================

import pandas as pd
import numpy as np
import joblib

from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier # Correctly import XGBClassifier

from sklearn.metrics import (
    accuracy_score, f1_score, precision_score,
    recall_score, roc_auc_score, classification_report
)

# ============== CONFIG ==============
# Use the DATA_PATH variable defined earlier in the notebook
# DATA_PATH = "final_dataset_realistic 1.csv"  # adjust if needed
RANDOM_STATE = 42
N_SPLITS = 5
TEST_SIZE = 0.2

# ============== LOAD DATA ==============
# Use the DATA_PATH variable to load the data
df = pd.read_csv(DATA_PATH)
df.columns = df.columns.str.strip()

if "Problem Type" not in df.columns:
    raise ValueError("Dataset must contain 'Problem Type' column.")

# Clean problem type
df["Problem Type"] = df["Problem Type"].astype(str).str.lower().str.strip()

# Identify target column (any column containing 'readmit' or 'readmission')
target_candidates = [c for c in df.columns
                     if "readmit" in c.lower() or "readmission" in c.lower()]
if not target_candidates:
    raise ValueError("No readmission target column found.")
target_col = target_candidates[0]

# Map target to 0/1
df[target_col] = (
    df[target_col]
    .astype(str).str.lower().str.strip()
    .map({"yes": 1, "true": 1, "1": 1, "no": 0, "false": 0, "0": 0})
    .fillna(0)
    .astype(int)
)

# ============== SUBSETS ==============
df_diabetes = df[df["Problem Type"].str.contains("diab", na=False)]
df_heart = df[df["Problem Type"].str.contains("heart", na=False)]

print(f"Diabetes rows: {df_diabetes.shape[0]}")
print(f"Heart Failure rows: {df_heart.shape[0]}")

if df_diabetes.empty:
    raise ValueError("No Diabetes records found. Check 'Problem Type' values.")
if df_heart.empty:
    raise ValueError("No Heart Failure records found. Check 'Problem Type' values.")

# ============== TRAINING FUNCTION ==============
def train_for_disease(df_sub: pd.DataFrame, disease_label: str, out_name_suffix: str):
    """
    Trains 3 models (LR, RF, XGB) for a given disease subset.
    Uses advanced preprocessing & hyperparameter tuning.
    Saves RandomForest as final .pkl.
    """
    print("\n" + "=" * 70)
    print(f"🏥 TRAINING FOR: {disease_label.upper()}")
    print("=" * 70)

    y = df_sub[target_col]
    X = df_sub.drop(columns=[target_col, "Problem Type"], errors="ignore")

    # Identify numeric & categorical columns
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    cat_cols = X.select_dtypes(include=["object", "bool"]).columns.tolist()

    # Preprocessor: impute + scale + one-hot
    preprocessor = ColumnTransformer(
        transformers=[
            ("num", Pipeline(steps=[
                ("imputer", SimpleImputer(strategy="median")),
                ("scaler", StandardScaler())
            ]), num_cols),
            ("cat", Pipeline(steps=[
                ("imputer", SimpleImputer(strategy="most_frequent")),
                ("onehot", OneHotEncoder(handle_unknown="ignore"))
            ]), cat_cols),
        ],
        remainder="drop"
    )

    # Define models
    models = {
        "LogisticRegression": LogisticRegression(
            max_iter=4000,
            solver="liblinear",
            class_weight="balanced",
            random_state=RANDOM_STATE,
        ),
        "RandomForest": RandomForestClassifier(
            random_state=RANDOM_STATE,
            class_weight="balanced",
            n_jobs=-1,
        ),
        "XGBoost": XGBClassifier(
            random_state=RANDOM_STATE,
            eval_metric="logloss",
            use_label_encoder=False,
            n_jobs=-1,
            # handle imbalance
            scale_pos_weight=max(
                1.0,
                float((y == 0).sum()) / max((y == 1).sum(), 1)
            ),
        ),
    }

    # Hyperparameter grids (tuned to push AUC/accuracy high)
    param_grids = {
        "LogisticRegression": {
            "clf__C": [0.01, 0.1, 1, 5, 10],
            "clf__penalty": ["l1", "l2"],
        },
        "RandomForest": {
            "clf__n_estimators": [200, 400, 600],
            "clf__max_depth": [10, 15, 20, None],
            "clf__min_samples_split": [2, 4],
            "clf__min_samples_leaf": [1, 2],
            "clf__max_features": ["sqrt", "log2"],
        },
        "XGBoost": {
            "clf__learning_rate": [0.01, 0.05, 0.1],
            "clf__max_depth": [3, 5, 7],
            "clf__n_estimators": [150, 250, 350],
            "clf__subsample": [0.8, 1.0],
            "clf__colsample_bytree": [0.8, 1.0],
        },
    }

    # Train/val split
    X_train, X_test, y_train, y_test = train_test_split(
        X, y,
        test_size=TEST_SIZE,
        stratify=y,
        random_state=RANDOM_STATE
    )

    cv = StratifiedKFold(
        n_splits=N_SPLITS, shuffle=True, random_state=RANDOM_STATE
    )

    results = []

    for name, base_clf in models.items():
        print("\n" + "-" * 50)
        print(f"🔹 {disease_label} - {name}")
        print("-" * 50)

        pipe = Pipeline(steps=[
            ("pre", preprocessor),
            ("clf", base_clf),
        ])

        grid = GridSearchCV(
            estimator=pipe,
            param_grid=param_grids[name],
            scoring="roc_auc",
            n_jobs=-1,
            cv=cv,
            verbose=1,
        )
        grid.fit(X_train, y_train)

        best = grid.best_estimator_
        y_pred = best.predict(X_test)
        y_prob = best.predict_proba(X_test)[:, 1]

        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        prec = precision_score(y_test, y_pred)
        rec = recall_score(y_test, y_pred)
        auc = roc_auc_score(y_test, y_prob)

        print(f"Best Params: {grid.best_params_}")
        print(f"Accuracy:      {acc:.4f}")
        print(f"Precision:     {prec:.4f}")
        print(f"Recall:        {rec:.4f}")
        print(f"F1 Score:      {f1:.4f}")
        print(f"ROC-AUC:       {auc:.4f}")
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))

        results.append({
            "Model": name,
            "Estimator": best,
            "Accuracy": acc,
            "F1": f1,
            "AUC": auc,
        })

    # Summarize results
    results_df = pd.DataFrame(results).sort_values(by=["AUC", "Accuracy"], ascending=False)
    print("\n" + "=" * 50)
    print(f"📊 SUMMARY - {disease_label}")
    print(results_df[["Model", "Accuracy", "F1", "AUC"]])
    print("=" * 50)

    # ----- Select Random Forest as final model -----
    # Check if RandomForest is in results before trying to access it
    rf_row = next((r for r in results if r["Model"] == "RandomForest"), None)

    if rf_row:
        rf_acc = rf_row["Accuracy"]
        rf_auc = rf_row["AUC"]

        if rf_acc < 0.80:
            print(f"\n Warning: RandomForest accuracy for {disease_label} is {rf_acc:.2%} (< 80%).")
            print("   Tune data/labels or features if you must strictly exceed 80%.")
        else:
            print(f"\n✅ RandomForest for {disease_label} achieved {rf_acc:.2%} accuracy (AUC={rf_auc:.3f}).")

        final_model = rf_row["Estimator"]
        out_path = f"readmission_{out_name_suffix}_RandomForest.pkl"
        joblib.dump(final_model, out_path)
        print(f"Saved RandomForest model for {disease_label} -> {out_path}")
    else:
        print(f"\n Warning: RandomForest model not found in results for {disease_label}. Skipping save.")


    return results_df

# ============== RUN TRAINING FOR BOTH ==============
diab_results = train_for_disease(df_diabetes, "Diabetes", "diabetes")
heart_results = train_for_disease(df_heart, "Heart Failure", "heart_failure")

print("\n================= ALL DONE =================")
print("Diabetes models summary:")
print(diab_results)
print("\nHeart Failure models summary:")
print(heart_results)
print("RandomForest PKL files are ready for your Flask app.")

Diabetes rows: 2488
Heart Failure rows: 2512

🏥 TRAINING FOR: DIABETES

--------------------------------------------------
🔹 Diabetes - LogisticRegression
--------------------------------------------------
Fitting 5 folds for each of 10 candidates, totalling 50 fits
Best Params: {'clf__C': 0.1, 'clf__penalty': 'l1'}
Accuracy:      0.6325
Precision:     0.5023
Recall:        0.6033
F1 Score:      0.5481
ROC-AUC:       0.6751

Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.65      0.69       314
           1       0.50      0.60      0.55       184

    accuracy                           0.63       498
   macro avg       0.62      0.63      0.62       498
weighted avg       0.65      0.63      0.64       498


--------------------------------------------------
🔹 Diabetes - RandomForest
--------------------------------------------------
Fitting 5 folds for each of 96 candidates, totalling 480 fits
Best Params: {'clf__max_depth'

In [27]:
!pip install xgboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.9/115.9 MB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 1.2 MB/s eta 0:00:00


MODEL SELECTION

In [42]:
print("\n--- Model selection by ROC-AUC ---")

# Combine the results from both training runs into the results dictionary
results = {}
for index, row in diab_results.iterrows():
    # Include all relevant metrics from the results DataFrames
    results[row['Model'] + "_Diabetes"] = {
        'roc_auc': row['AUC'],
        'accuracy': row['Accuracy'],
        'f1': row['F1'],
        'precision': None, # Precision is not directly available in diab_results summary, will need re-evaluation or storing more metrics
        'recall': None,    # Recall is not directly available
        'pr_auc': None     # PR-AUC is not directly available
    }
for index, row in heart_results.iterrows():
    # Include all relevant metrics from the results DataFrames
     results[row['Model'] + "_HeartFailure"] = {
        'roc_auc': row['AUC'],
        'accuracy': row['Accuracy'],
        'f1': row['F1'],
        'precision': None, # Precision is not directly available in heart_results summary
        'recall': None,    # Recall is not directly available
        'pr_auc': None     # PR-AUC is not directly available
    }

# --- FIX: Re-evaluate the best model on the test set to get all metrics for the summary ---
# Get the best pipeline (already done in cell u-mG2H9RfPCm and used in 5onZPExzgdqN)
# Assuming new_pipeline (trained on full X_train) is the one we want to summarize metrics for on X_test
if 'new_pipeline' in locals(): # Check if new_pipeline exists from previous execution
    print(f"Evaluating the retrained pipeline ({best_name.split('_')[0]} on full data) on X_test for summary metrics...")
    y_prob_test_summary = new_pipeline.predict_proba(X_test)[:,1]
    y_pred_test_summary = (y_prob_test_summary >= RISK_THRESHOLD).astype(int)

    # Update the results dictionary for the selected best model with metrics from the evaluation on X_test
    # Note: This assumes the best_name corresponds to the model structure in new_pipeline
    # This is a simplification, ideally, we'd store full metrics during the grid search
    # or re-evaluate the specific best model from diab_results/heart_results on its subset test set.
    # Given the flow, summarizing the 'new_pipeline' (best structure on full data) on X_test is most practical.
    if best_name in results:
         results[best_name]['roc_auc'] = roc_auc_score(y_test, y_prob_test_summary)
         results[best_name]['pr_auc'] = average_precision_score(y_test, y_prob_test_summary)
         results[best_name]['accuracy'] = accuracy_score(y_test, y_pred_test_summary)
         results[best_name]['precision'] = precision_score(y_test, y_pred_test_summary, zero_division=0)
         results[best_name]['recall'] = recall_score(y_test, y_pred_test_summary, zero_division=0)
         results[best_name]['f1'] = f1_score(y_test, y_pred_test_summary, zero_division=0)
         # Confusion matrix is not easily stored in this summary structure, skipping

    print("Updated summary metrics for:", best_name)
else:
    print("Warning: new_pipeline not found. Summary metrics in JSON might be incomplete.")
# --- End of FIX ---


best_name = max(results.items(), key=lambda kv: kv[1]['roc_auc'])[0]
print("Model scores (ROC-AUC):")
for k,v in results.items():
    # Print available metrics
    metrics_str = f"ROC-AUC={v.get('roc_auc', 'N/A'):.4f}"
    if v.get('accuracy') is not None: metrics_str += f", Acc={v['accuracy']:.4f}"
    if v.get('f1') is not None: metrics_str += f", F1={v['f1']:.4f}"
    if v.get('precision') is not None: metrics_str += f", Prec={v['precision']:.4f}"
    if v.get('recall') is not None: metrics_str += f", Rec={v['recall']:.4f}"
    if v.get('pr_auc') is not None: metrics_str += f", PR-AUC={v['pr_auc']:.4f}"
    print(f" - {k}: {metrics_str}")

print("Selected best model:", best_name)

# This part of the code needs to be adjusted based on how 'best_pipeline' is intended to be used.
# The previous training cell saved individual models for Diabetes and Heart Failure.
# This cell was originally set up to select one best pipeline from a single training run.
# Since we trained two separate models (RF for Diabetes and RF for Heart Failure),
# the concept of a single "best_pipeline" for the entire dataset might not be appropriate here.
# If the goal is to select the single best *Random Forest* model across both diseases,
# we would need to determine which one had a higher ROC-AUC during its specific training run.

# For now, let's assume we want to select the overall best model based on ROC-AUC from the combined results
# However, remember these AUCs are from separate test sets (Diabetes test set and Heart Failure test set).
# A more robust approach would be to evaluate ALL trained models (LR, RF, XGB for both diseases)
# on the full test set X_test from the original split, after applying the appropriate preprocessing.

# Given the current structure, the original logic of selecting a single best_pipeline based on the 'results' dictionary
# does not align with the two separate saved models.

# I will comment out the saving part for now as it requires a decision on which model to save as "best".
# best_model_path = os.path.join(OUTPUT_DIR, "best_readmission_pipeline.joblib")
# joblib.dump(best_pipeline, best_model_path)
# print("Saved best pipeline to:", best_model_path)

# Note: The downstream cells (Explanaibility, Score Full Data, Plot ROC/PR, Interpretible Risk Mapping)
# will also need to be adapted to work with either the separate disease-specific models,
# or by selecting and loading one of the saved models (e.g., the best overall performing one or a specific one like RF for Diabetes).


--- Model selection by ROC-AUC ---
Evaluating the retrained pipeline (LogisticRegression on full data) on X_test for summary metrics...
Updated summary metrics for: LogisticRegression_HeartFailure
Model scores (ROC-AUC):
 - LogisticRegression_Diabetes: ROC-AUC=0.6751, Acc=0.6325, F1=0.5481
 - RandomForest_Diabetes: ROC-AUC=0.6680, Acc=0.6466, F1=0.1538
 - XGBoost_Diabetes: ROC-AUC=0.6666, Acc=0.6446, F1=0.5630
 - LogisticRegression_HeartFailure: ROC-AUC=0.6398, Acc=0.6050, F1=0.4981, Prec=0.4569, Rec=0.5475, PR-AUC=0.5036
 - XGBoost_HeartFailure: ROC-AUC=0.6773, Acc=0.6302, F1=0.5303
 - RandomForest_HeartFailure: ROC-AUC=0.6399, Acc=0.6620, F1=0.3841
Selected best model: XGBoost_HeartFailure


EXPLANABILITY

In [32]:
print("\nComputing feature names (post-preprocessing)...")

# --- FIX: Retrieve the best pipeline based on best_name ---
# Find the best model's estimator from the results DataFrames
best_pipeline = None
model_name, disease_suffix = best_name.rsplit('_', 1) # Split 'LogisticRegression_HeartFailure' into 'LogisticRegression' and 'HeartFailure'

if disease_suffix == 'Diabetes':
    best_row = diab_results[diab_results['Model'] == model_name]
    if not best_row.empty:
        best_pipeline = best_row.iloc[0]['Estimator']
elif disease_suffix == 'HeartFailure':
    best_row = heart_results[heart_results['Model'] == model_name]
    if not best_row.empty:
        best_pipeline = best_row.iloc[0]['Estimator']

if best_pipeline is None:
    raise RuntimeError(f"Could not find the estimator for the best model: {best_name}")
# --- End of FIX ---


# try to reconstruct feature names
feature_names = []
try:
    pre = best_pipeline.named_steps['pre']
    # ColumnTransformer.get_feature_names_out is available on modern sklearn
    try:
        feature_names = pre.get_feature_names_out()
    except Exception:
        # attempt manual concatenation
        # Need to get the correct original columns used by the preprocessor
        # This is tricky as the original X varies by disease subset
        # A safer approach might be to fit the preprocessor on the full X_train
        # and then use its get_feature_names_out.
        # For now, let's try to get the columns from the preprocessor pipeline steps if possible
        num_cols_processed = pre.transformers_[0][2] if len(pre.transformers_) > 0 and pre.transformers_[0][0] == 'num' else []
        cat_cols_processed = pre.transformers_[1][2] if len(pre.transformers_) > 1 and pre.transformers_[1][0] == 'cat' else []

        cat_encoder = pre.named_transformers_['cat'].named_steps['onehot'] if 'cat' in pre.named_transformers_ else None
        if cat_encoder is not None:
            # Use cat_cols_processed which are the original column names passed to the categorical transformer
            cat_names = cat_encoder.get_feature_names_out(cat_cols_processed)
            feature_names = list(num_cols_processed) + list(cat_names)
        else:
            feature_names = list(num_cols_processed) + list(cat_cols_processed) # Fallback if onehot is not found


except Exception as e:
    print("Could not extract feature names automatically:", e)
    # Fallback to original columns if feature name extraction fails
    # Note: This will not match the post-preprocessing features but prevents error
    feature_names = list(X.columns)


# Feature importances for tree models
clf = best_pipeline.named_steps.get('clf', None)
if hasattr(clf, "feature_importances_"):
    try:
        importances = clf.feature_importances_
        # align length
        if len(feature_names) == len(importances):
             imp_df = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False)
             imp_df.to_csv(os.path.join(OUTPUT_DIR, "feature_importances.csv"), index=False)
             print("Saved feature importances to output directory.")
        else:
            print(f"Feature names count ({len(feature_names)}) does not match importances count ({len(importances)}); skipping importance save.")

    except Exception as e:
        print("Could not produce feature importances:", e)
elif hasattr(clf, "coef_"):
     # For linear models like Logistic Regression, use coefficients
     try:
        coefs = clf.coef_[0] if clf.coef_.ndim > 1 else clf.coef_
        # Ensure feature_names length matches coefficients length
        if len(feature_names) == len(coefs):
            coef_df = pd.DataFrame({'feature': feature_names, 'coefficient': coefs}).sort_values('coefficient', ascending=False)
            coef_df.to_csv(os.path.join(OUTPUT_DIR, "feature_coefficients.csv"), index=False)
            print("Saved feature coefficients to output directory.")
        else:
             print(f"Feature names count ({len(feature_names)}) does not match coefficients count ({len(coefs)}); skipping coefficient save.")

     except Exception as e:
        print("Could not produce feature coefficients:", e)

else:
    print("Best model does not expose feature_importances_ or coef_.")

# SHAP explanations (if available)
if SHAP_AVAILABLE:
    try:
        print("Computing SHAP values (sample) — this can take time...")
        # generate background by sampling training set after preprocessing
        # Use best_pipeline to transform
        preproc = best_pipeline.named_steps['pre']
        # Take a small background from the relevant training set subset
        if disease_suffix == 'Diabetes':
            X_train_subset = X_train[X_train['Problem Type'].str.contains("diab", na=False)].drop(columns=["Problem Type"], errors="ignore")
        elif disease_suffix == 'HeartFailure':
             X_train_subset = X_train[X_train['Problem Type'].str.contains("heart", na=False)].drop(columns=["Problem Type"], errors="ignore")
        else:
             X_train_subset = X_train.drop(columns=["Problem Type"], errors="ignore") # Fallback to full X_train if disease is unknown

        if not X_train_subset.empty:
            bg = X_train_subset.sample(min(200, len(X_train_subset)), random_state=RANDOM_STATE)
            X_bg = preproc.transform(bg)

            # Prepare sample for SHAP from the relevant test set subset
            if disease_suffix == 'Diabetes':
                X_test_subset = X_test[X_test['Problem Type'].str.contains("diab", na=False)].drop(columns=["Problem Type"], errors="ignore")
            elif disease_suffix == 'HeartFailure':
                 X_test_subset = X_test[X_test['Problem Type'].str.contains("heart", na=False)].drop(columns=["Problem Type"], errors="ignore")
            else:
                 X_test_subset = X_test.drop(columns=["Problem Type"], errors="ignore") # Fallback to full X_test if disease is unknown

            if not X_test_subset.empty:
                sample_for_shap_df = X_test_subset.sample(min(100, len(X_test_subset)), random_state=RANDOM_STATE)
                sample_for_shap = preproc.transform(sample_for_shap_df)

                # create shap explainer for estimator
                explainer = shap.Explainer(best_pipeline.named_steps['clf'], X_bg, feature_names=feature_names)
                shap_vals = explainer(sample_for_shap)
                # save summary plot
                shap.summary_plot(shap_vals, feature_names=feature_names, show=False)
                plt.savefig(os.path.join(OUTPUT_DIR, f"shap_summary_{disease_suffix}.png"), bbox_inches='tight')
                plt.close()
                print(f"Saved SHAP summary plot for {disease_suffix}.")
            else:
                 print(f"Test subset for {disease_suffix} is empty; skipping SHAP for this subset.")
        else:
            print(f"Train subset for {disease_suffix} is empty; skipping SHAP for this subset.")

    except Exception as e:
        print("SHAP failed:", e)
else:
    print("SHAP not installed; skip SHAP analysis.")


Computing feature names (post-preprocessing)...
Saved feature coefficients to output directory.
SHAP not installed; skip SHAP analysis.


SCORE FULL DATA

In [38]:
print("\nScoring full dataset and saving predictions...")
X_full = X.copy()

# --- FIX: Create a new pipeline with a preprocessor configured for full data and train on X_train ---

# 1. Get the best classifier's type and hyperparameters from the best_pipeline
best_classifier_type = type(best_pipeline.named_steps['clf'])
best_classifier_params = best_pipeline.named_steps['clf'].get_params()

# 2. Create a new preprocessor configured for the full dataset using numeric_cols and cat_cols
# These variables were defined in cell eORIP4-VHQcG and should be available.
full_data_preprocessor = ColumnTransformer(transformers=[
    ('num', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='median')),
        ('scaler', StandardScaler())
    ]), numeric_cols), # Use numeric_cols from the full dataset analysis
    ('cat', Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ]), cat_cols) # Use cat_cols from the full dataset analysis
], remainder='drop', sparse_threshold=0)

# 3. Create a new pipeline with the full data preprocessor and the best classifier
new_pipeline = Pipeline(steps=[
    ('pre', full_data_preprocessor),
    ('clf', best_classifier_type(**best_classifier_params)) # Instantiate the best classifier with its params
])

# 4. Fit this new pipeline on the original full training data (X_train, y_train)
print(f"Fitting a new pipeline with the best classifier structure ({best_name.split('_')[0]}) on the full training data (X_train, y_train)...")
new_pipeline.fit(X_train, y_train)
print("Fitting complete.")

# 5. Use this newly fitted pipeline to predict on the full X_full data
pred_probs = new_pipeline.predict_proba(X_full)[:,1]
# --- End of FIX ---

df_out = df_fe.copy()
df_out['Predicted_Risk_Score'] = pred_probs
df_out['Predicted_Readmission'] = np.where(df_out['Predicted_Risk_Score'] >= RISK_THRESHOLD, 'Yes', 'No')

out_csv = os.path.join(OUTPUT_DIR, "readmission_predictions_with_features.csv")
df_out.to_csv(out_csv, index=False)
print("Saved predictions CSV to:", out_csv)


Scoring full dataset and saving predictions...
Fitting a new pipeline with the best classifier structure (LogisticRegression) on the full training data (X_train, y_train)...
Fitting complete.
Saved predictions CSV to: /content/drive/MyDrive/readmission_output.csv/readmission_predictions_with_features.csv


PLOT ROC AND PR CURVES

In [40]:
print("Plotting ROC and PR curves for best model on test set...")
# --- FIX: Use the new_pipeline (trained on full X_train) to predict on X_test ---
# The new_pipeline was created and fitted in the previous cell (UB2qbX7ngQnc)
# Assuming new_pipeline is available from the previous execution

y_prob_test = new_pipeline.predict_proba(X_test)[:,1]
# --- End of FIX ---

fpr, tpr, _ = roc_curve(y_test, y_prob_test)
precision, recall, _ = precision_recall_curve(y_test, y_prob_test)
roc_auc_val = roc_auc_score(y_test, y_prob_test)
pr_auc_val = average_precision_score(y_test, y_prob_test)

plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, label=f'ROC (AUC={roc_auc_val:.3f})')
plt.plot([0,1],[0,1],'--', color='gray')
plt.xlabel('False Positive Rate'); plt.ylabel('True Positive Rate'); plt.title('ROC Curve'); plt.legend(); plt.grid(True)
plt.savefig(os.path.join(OUTPUT_DIR, "roc_curve.png"))
plt.close()

plt.figure(figsize=(6,5))
plt.plot(recall, precision, label=f'PR (AUC={pr_auc_val:.3f})')
plt.xlabel('Recall'); plt.ylabel('Precision'); plt.title('Precision-Recall Curve'); plt.legend(); plt.grid(True)
plt.savefig(os.path.join(OUTPUT_DIR, "pr_curve.png"))
plt.close()

print("Saved ROC and PR plots to output directory.")

Plotting ROC and PR curves for best model on test set...
Saved ROC and PR plots to output directory.


CREATE INTERPRETABLE RISK SCORING SYSTEM MAPPING

In [44]:
def risk_group(score):
    if score > 0.7:
        return "HIGH"
    if score > 0.3:
        return "MODERATE"
    return "LOW"

df_out['RiskGroup'] = df_out['Predicted_Risk_Score'].apply(risk_group)
df_out.to_csv(out_csv, index=False)  # overwrite with risk group included
print("Added RiskGroup and updated CSV:", out_csv)

# Save a JSON summary of metrics
# --- FIX: Use metrics from the evaluation of new_pipeline on X_test ---
# Assuming roc_auc_val, pr_auc_val, and other metrics from evaluating new_pipeline on X_test
# are available from previous cell execution (e.g., cell 5onZPExzgdqN)
# We also need accuracy, precision, recall, and f1 from that evaluation.
# Since new_pipeline was evaluated on X_test in cell 5onZPExzgdqN, we can re-calculate
# the classification metrics here using y_test and predictions from new_pipeline.
# Alternatively, ensure the evaluation in cell 5onZPExzgdqN saves these metrics to variables.
# Let's assume y_test and new_pipeline are available and re-calculate here for robustness.

if 'new_pipeline' in locals() and 'y_test' in locals():
    # FIX: Pass X_test instead of y_test.index to predict_proba
    y_prob_test_for_summary = new_pipeline.predict_proba(X_test)[:, 1]
    y_pred_test_for_summary = (y_prob_test_for_summary >= RISK_THRESHOLD).astype(int)

    summary = {
        'best_model': best_name,
        'metrics_test': {
            'roc_auc': roc_auc_score(y_test, y_prob_test_for_summary),
            'pr_auc': average_precision_score(y_test, y_prob_test_for_summary),
            'accuracy': accuracy_score(y_test, y_pred_test_for_summary),
            'precision': precision_score(y_test, y_pred_test_for_summary, zero_division=0),
            'recall': recall_score(y_test, y_pred_test_for_summary, zero_division=0),
            'f1': f1_score(y_test, y_pred_test_for_summary, zero_division=0)
        },
        'timestamp': datetime.utcnow().isoformat()
    }
    print("Populated summary metrics using new_pipeline evaluation on X_test.")
else:
     # Fallback if new_pipeline or y_test are not available (shouldn't happen with current flow)
     summary = {
        'best_model': best_name,
        'metrics_test': {
            'roc_auc': results[best_name].get('roc_auc'),
            'pr_auc': results[best_name].get('pr_auc'),
            'accuracy': results[best_name].get('accuracy'),
            'precision': results[best_name].get('precision'),
            'recall': results[best_name].get('recall'),
            'f1': results[best_name].get('f1')
        },
        'timestamp': datetime.utcnow().isoformat()
    }
     print("Warning: new_pipeline or y_test not found. Populating summary from potentially incomplete results dict.")

# --- End of FIX ---

with open(os.path.join(OUTPUT_DIR, "training_summary.json"), "w") as f:
    json.dump(summary, f, indent=2)

print("\nAll done. Artifacts written to:", OUTPUT_DIR)
# Safely print metrics from the summary dictionary
print("Best model:", summary.get('best_model'))
metrics = summary.get('metrics_test', {})
print(f"Metrics (Test Set): ROC-AUC={metrics.get('roc_auc', 'N/A'):.4f}, PR-AUC={metrics.get('pr_auc', 'N/A'):.4f}, Acc={metrics.get('accuracy', 'N/A'):.4f}, F1={metrics.get('f1', 'N/A'):.4f}")

Added RiskGroup and updated CSV: /content/drive/MyDrive/readmission_output.csv/readmission_predictions_with_features.csv
Populated summary metrics using new_pipeline evaluation on X_test.

All done. Artifacts written to: /content/drive/MyDrive/readmission_output.csv
Best model: XGBoost_HeartFailure
Metrics (Test Set): ROC-AUC=0.6398, PR-AUC=0.5036, Acc=0.6050, F1=0.4981
